In [ ]:
%pip install tqdm
%pip install numpy
%pip install sentencepiece protobuf
%pip install python-dotenv==1.2.1
%pip install openai==2.16.0
%pip install azure-ai-inference==1.0.0b9
%pip install sentence-transformers==5.2.2 tiktoken==0.12.0
%pip install fastembed==0.7.4

%restart_python

In [ ]:
import os
import json
import asyncio
import numpy  as np
import scipy  as sp
from typing        import List
from dotenv        import load_dotenv
from logging       import Logger, getLogger
from openai        import OpenAI, AsyncOpenAI
from tqdm.notebook import tqdm
from tqdm.asyncio  import tqdm_asyncio

import pandas as pd
from pyspark.sql import SparkSession
from delta       import configure_spark_with_delta_pip

from azure.ai.inference.models import SystemMessage, UserMessage
from sentence_transformers     import SentenceTransformer
from fastembed                 import TextEmbedding


In [ ]:
builder = SparkSession.builder\
            .config("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")\
            .config("spark.sql.parquet.output.committer.class", "org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter")\
            .config("spark.mapreduce.fileoutputcommitter.marksuccessfuljobs","false")\
            .config("spark.sql.adaptive.enabled", True)\
            .config("spark.sql.shuffle.partitions", "auto")\
            .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "100MB")\
            .config("spark.sql.adaptive.coalescePartitions.enabled", True)\
            .config("spark.sql.dynamicPartitionPruning.enabled", True)\
            .config("spark.sql.autoBroadcastJoinThreshold", "10MB")\
            .config("spark.sql.session.timeZone", "Asia/Tokyo")\
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .config("spark.databricks.delta.write.isolationLevel", "SnapshotIsolation")\
            .config("spark.databricks.delta.optimizeWrite.enabled", True)\
            .config("spark.databricks.delta.autoCompact.enabled", True)
            # Delta Lake 用の SQL コミットプロトコルを指定
            # Parquet 出力時のコミッタークラスを指定
            # Azure Blob Storage (ABFS) 用のコミッターファクトリを指定
            # '_SUCCESS'で始まるファイルを書き込まないように設定
            # AQE(Adaptive Query Execution)の有効化
            # パーティション数を自動で調整するように設定
            # シャッフル後の1パーティションあたりの最小サイズを指定
            # AQEのパーティション合成の有効化
            # 動的パーティションプルーニングの有効化
            # 小さいテーブルのブロードキャスト結合への自動変換をするための閾値調整
            # SparkSessionのタイムゾーンを日本標準時刻に設定
            # Delta Lake固有のSQL構文や解析機能を拡張モジュールとして有効化
            # SparkカタログをDeltaLakeカタログへ変更
            # Delta Lake書き込み時のアイソレーションレベルを「スナップショット分離」に設定
            # 書き込み時にデータシャッフルを行い、大きなファイルを生成する機能の有効化
            # 書き込み後に小さなファイルを自動で統合する機能の有効化


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
# .env ファイルを読み込む
load_dotenv()

# 環境変数の取得
AI_FOUNDRY_ENDPOINT = os.environ.get("AI_FOUNDRY_ENDPOINT")
AI_FOUNDRY_API_KEY  = os.environ.get("AI_FOUNDRY_API_KEY")
AI_FOUNDRY_MODEL    = os.environ.get("AI_FOUNDRY_MODEL")
MAX_TOKENS          = os.environ.get("MAX_TOKENS")
TEMPERATURE         = os.environ.get("TEMPERATURE")
TOP_P               = os.environ.get("TOP_P")

# メモ：
# ウィジット経由でのパラメータの取得方法では、無条件にパラメータの型がStringに変換されてしまう
# そのため、受け取ったパラメータを適切に型変換する必要がある
MAX_TOKENS  = int(MAX_TOKENS)
TEMPERATURE = float(TEMPERATURE)
TOP_P       = float(TOP_P)


# 簡易デバッグ用
print(f'AI_FOUNDRY_ENDPOINT: {AI_FOUNDRY_ENDPOINT}')
print(f'AI_FOUNDRY_API_KEY:  {AI_FOUNDRY_API_KEY}')
print(f'AI_FOUNDRY_MODEL:    {AI_FOUNDRY_MODEL}')
print(f'MAX_TOKENS:          {MAX_TOKENS}')
print(f'TEMPERATURE:         {TEMPERATURE}')
print(f'TOP_P:               {TOP_P}')

# 場所の文脈ベクトル化：アプローチの進化と実装課題

## 1. 目的と課題 (Objective & Challenge)
- **目的**: 広告LP (Landing Page) と物理的な「場所」を高次元でマッチングさせ、高精度な広告配信を実現する。
- **課題**: 単なる「場所名・施設名」の埋め込みでは、その場所が持つ**「雰囲気・客層・利用文脈」といった深い意味空間（Context）**を捉えきれず、マッチング精度が頭打ちになる。

## 2. アプローチの変遷 (Evolution)
当初の「LLMによる一般的な想像」から、実データであるADID（広告ID）に基づく「行動からの逆算」へと方針を転換した。

| 比較項目 | 初期案：LLMによる想像 (Generative) | **修正案：ADID行動解析 (Bayesian Approach)** |
| :--- | :--- | :--- |
| **アプローチ** | **Top-down (演繹的)**<br>場所名から一般的なイメージを降ろしてくる手法。 | **Bottom-up (帰納的)**<br>個々の「来訪者(ADID)」の集合から、場所の意味を積み上げる手法。 |
| **情報の源泉** | **LLMの内部知識 (Frozen Weights)**<br>学習データに含まれる過去の知識や通説。 | **物理行動ログ (Observed Data)**<br>実際にその場所に足を運んだADIDの移動履歴という「事実」。 |
| **文脈の解像度** | **ステレオタイプ (Stereotypical)**<br>画一的なペルソナになりがち。 | **実態の反映 (Empirical)**<br>場所ごとの固有の客層・文脈を捉える。 |
| **情報の鮮度** | **静的 (Static)**<br>モデルの学習時期に依存。 | **動的 (Dynamic)**<br>直近のログを使用し、トレンドや季節性を反映可能。 |

## 3. 理論モデル：ベイズ推定と行動ログによる文脈抽出

「場所の意味（Context）」を、静的な属性ではなく、**そこに集まる個々のエージェント（ADID）が持つ目的意識の期待値**として定義する。

### 3.1 変数定義
- $L$: Location（対象となる場所・施設）
- $A$: Agent / ADID（個々のユーザー）
- $M$: Meaning / Context（場所が持つ意味空間）
- $\mathbf{v}$: 埋め込みベクトル空間（Embedding Space, $\mathbb{R}^d$）

### 3.2 ユーザー分布の事後確率推定
まず、観測された行動ログから、ある場所 $L$ に特定のユーザー $A$ が存在する事後確率 $P(A|L)$ を求める。

$$
P(A | L) \propto P(L | A) P(A)
$$

### 3.3 個別文脈の生成：多面的推論と平均ベクトル (Multi-Hypothesis Embedding)
特定のユーザー $A$ が場所 $L$ にいるときの「意味」を一意に決めるのではなく、**Unity Catalog (Delta Lake)** に格納されたリッチな属性データに基づき、LLMに多様な可能性を推論させる。

#### (1) 入力データの構築 (Feature Construction)
ユーザー $A$ を定義するプロンプト $\mathbf{x}_A$ を、以下のテーブル群から構成する：

$$
\mathbf{x}_A \leftarrow \{ \mathbf{d}_{\text{geo}}, \mathbf{d}_{\text{demo}}, \mathbf{d}_{\text{cohort}}, \mathbf{d}_{\text{hist}} \}
$$

- **`relational_address_jp`**: 居住地・勤務地（生活圏の特定）
- **`mobilewalla_agegender`, `mobaku_agegender`**: 性別・年代（デモグラフィック属性）
- **`relational_cohortlist`**: 生活行動・興味関心（サイコグラフィック属性）
- **`relational_spot`**: 過去の行動履歴（コンテキストの補強）

#### (2) 多面的推論 (Generative Inference)
LLMに対し、「この属性を持つ $A$ が 場所 $L$ にいる目的と行動」について、**$K$ 個の仮説（Hypotheses）**を生成させる（例: $K \approx 20$）。

$$
H_{A,L} = \{ h_1, h_2, \dots, h_K \} \quad \text{where } h_i \sim \text{LLM}(\text{Prompt}(\mathbf{x}_A, L))
$$

#### (3) ベクトル化と集約 (Embedding & Aggregation)
生成された各仮説 $h_i$ を **SentenceTransformer ($\phi$)** を用いて埋め込みベクトルに変換し、それらの平均ベクトルを計算する。

$$
\mathbf{v}_{A,L} = \frac{1}{K} \sum_{i=1}^{K} \phi(h_i)
$$

### 3.4 場所の意味の周辺化 (Final Context Vector)
最終的な場所 $L$ の意味分布 $P(M | L)$ は、ユーザー $A$ に関して周辺化（Marginalize）することで得られる。
これをベクトル空間における**期待値**として解釈すると、場所の文脈ベクトル $\mathbf{v}_L$ は以下のように定式化される。

$$
\mathbf{v}_L = \sum_{A \in \mathcal{A}} \mathbf{v}_{A,L} \cdot P(A | L)
$$

これにより、単純な施設名の埋め込みではなく、**「ADIDの集合が、その場所でどのような目的を持って行動したか」**を表現する高精度なベクトルが得られる。

## 4. 実装の壁 (The Computational Wall)
上記のモデルは理論的に正しいが、物理的な計算リソースの限界に直面している。

### パラメータ規模
- **ADID数 ($N_A$)**: $\approx 1$億 ($10^8$)
- **場所数 ($N_L$)**: $\approx 2,500$ ($2.5 \times 10^3$)

### 課題
全ての組み合わせについて $P(M | A, L)$ をLLMで推論・計算しようとすると、計算量は $N_A \times N_L$ のオーダーとなる。

$$
10^8 \times 2,500 = 2.5 \times 10^{11} \quad (\text{2500億回のLLM推論})
$$

これはAPIコストおよび処理時間の観点で**天文学的なリソース**を必要とするため、全数計算は事実上不可能である。したがって、この理論モデルの精度を維持しつつ、計算量を劇的に削減する工夫（サンプリング、クラスタリング等）が必要となる。


## 5. 解決策：誘導点を用いた変分スパース近似 (Sparse Variational Approach with Inducing Points)

ガウス過程回帰 (GPR) における計算量削減テクニックである **「誘導点 (Inducing Points)」** の概念を応用する。

### 5.1 基本思想：Inducing Points (誘導点)
1億人のADID ($N$) を全て計算に使うのではなく、そのデータ空間全体を少数の「仮想的な代表点（誘導点 $Z$）」で近似表現する。

- **Full Data ($X$)**: 実際のADID特徴量（$N \approx 10^8$）。計算不能。
- **Inducing Points ($Z$)**: データ全体の分布を最適に記述できる「仮想的なユーザープロファイル」（$M \approx 20 \sim 50$）。

この $Z$ は、既存のADIDから選ぶのではなく、**変分下界 (ELBO) を最大化するように、特徴空間内で最適化（学習）された「最も説明力の高い仮想ペルソナ」** である。

### 5.2 アルゴリズム詳細

#### Step 1: 誘導点 $Z$ の配置最適化
場所 $L$ に存在する全ADIDの特徴量分布 $p(X|L)$ を近似するために、 $M$ 個の誘導点 $Z = \{z_1, \dots, z_M\}$ を配置する。

これは、変分法（Variational Inference）を用いて、実際のデータ $X$ と誘導点 $Z$ の間の情報損失（KL Divergence）を最小化する問題として定式化される。

$$
Z^* = \underset{Z}{\text{argmin}} \text{KL}[p(X|L) || q(X|Z)]
$$

これにより、「単なるクラスタリングの重心」よりも、**「データの分散や特異点をカバーしつつ、全体を疎 (Sparse) に表現できる最適な点」** が自動的に求まる。

#### Step 2: カーネル法による重み付け (Kernel Smoothing)
ある場所 $L$ にいる実際のユーザー群が、どの誘導点 $z_m$ に近いかを表す「親和度」をカーネル関数 $k(\cdot, \cdot)$ で計算する。

$$
w_m = \sum_{A \in L} k(\mathbf{x}_A, z_m)
$$

- $k(\mathbf{x}_A, z_m)$: 実際のユーザー $A$ の特徴ベクトルと、誘導点 $z_m$ の類似度。
- これにより、場所 $L$ が「どの誘導点（ペルソナ）の成分を強く持っているか」が重み $w$ として算出される。

#### Step 3: 誘導点に対するLLM推論 (Sparse Inference)
**LLMによる推論は、この $M$ 個の誘導点 $Z$ に対してのみ実行する。**

$$
\mathbf{v}_{z_m} = \text{Embed}(\text{LLM}(\text{Prompt}(z_m, L)))
$$

- **入力**: 最適化された仮想プロファイル $z_m$（例: 特徴空間上の座標としてのペルソナ）。
- **出力**: そのペルソナが場所 $L$ で持つ文脈ベクトル。
- **計算回数**: $N$ 回ではなく $M$ 回（$20 \sim 50$ 回）で済む。

#### Step 4: 文脈ベクトルの再構成 (Reconstruction)
最終的な場所 $L$ のベクトル $\mathbf{v}_L$ は、誘導点のベクトル $\mathbf{v}_{z_m}$ の線形結合として予測（回帰）される。

$$
\mathbf{v}_L \approx \sum_{m=1}^{M} \alpha_m \cdot \mathbf{v}_{z_m}
$$
(ここで $\alpha_m$ は Step 2 で求めた重み $w_m$ に基づく係数)

### 5.3 この手法の利点 (Why Sparse GP?)

1.  **計算コストの圧倒的削減**:
    - 計算オーダーは $O(N)$ から $O(M^2)$ または $O(NM)$ に下がる。$M$ は非常に小さいため（数十）、現実的な時間で計算可能。
2.  **外れ値への対応 (Robustness)**:
    - K-Meansのような単純平均では「外れ値」が重心をずらしてしまうが、Gaussian Processベースの誘導点はデータの「広がり（分散）」も考慮するため、ノイズに強い。
3.  **連続空間での補間**:
    - 誘導点は実在するADIDである必要がない。特徴空間上の「最適な中間地点」を仮想的に生成できるため、離散的なADIDだけでは表現しきれない「平均的なペルソナ」や「境界的なペルソナ」を表現できる。

### 6. 結論 (Summary)
**Sparse Variational approach (Inducing Points)** を採用することで、以下のパイプラインが完成する。

1.  Unity CatalogからADIDデータ ($10^8$) をロード。
2.  各場所について、データを代表する **少数の誘導点 ($Z$)** を変分学習で特定。
3.  その **数十個の誘導点だけ** をLLMに入力し、文脈ベクトルを生成。
4.  カーネル法を用いて元のデータ分布に射影し、場所のベクトルを合成。

これにより、理論的な正しさを保ったまま、天文学的な計算コストを現実的な範囲に収めることができる。


In [ ]:
# メモ：
# cohort.npzは以下のようなデータ構成になっている
# cohort.npz
#     |-- data              : 計算済みコホート係数行列
#     |-- indices           : データの位置指定子(列)
#     |-- indptr            : 行ごとのスライスインデックス
#     |-- shape             : コホート係数行列の形(ADIDのリスト × コホートキャプションのリスト)
#     |-- adid_list         : ADIDのリスト(列)
#     |-- business_codelist : コホートキャプションIDのリスト(行)
TARGET      = "/Volumes/stgadintedmpadintedi/featurestore/behaviorvector/cohort.npz"
npz         = np.load(TARGET, allow_pickle=True)
np_codelist = npz["business_codelist"]


# CODELISTに対するキャプションリストを取得
NAVIT_BUSINESS_TABLE = "adinte_datainfrastructure.list.navit_business"
sdf_navit_business   = spark.read.table(NAVIT_BUSINESS_TABLE)\
							.select(['BUSINESS_CODE', 'BUSINESS_NAME_S'])\
                            .join(spark.createDataFrame([(elem,) for elem in np_codelist.tolist()], ['BUSINESS_CODE']), on='BUSINESS_CODE', how='inner')

sdf_navit_business.toPandas().to_csv('cohort_caption_list.csv', index=False, header=True)
pdf_navit_business   = pd.read_csv('cohort_caption_list.csv', header=0, dtype={'BUSINESS_CODE': str, 'BUSINESS_NAME_S':str})
dict_code2name       = pdf_navit_business.set_index('BUSINESS_CODE')['BUSINESS_NAME_S'].to_dict()
dict_code2name

In [ ]:
llmClient  = AsyncOpenAI(base_url=AI_FOUNDRY_ENDPOINT, api_key=AI_FOUNDRY_API_KEY)
system_msg = SystemMessage(content=(
				"あなたは、特定の場所・施設名から、そこに集まる人々の輪郭を鮮明に描き出す高度な空間データアナリストです。\n"
                "それぞれの場所に対し、その空間の具体的な情景や特性、利用者の属性、感覚、行動パターンを深掘りし、以下のJSON形式で分析結果を出力してください。\n\n"

				"【目的】\n"
				"この出力は、単語の分散表現（文脈ベクトル）を生成するための学習データとして使用されます。\n"
    			"そのため、一般的・抽象的な説明ではなく、その場所に強く紐付く「具体的な名詞（物体・商品・設備）」や「動詞（動作）」を含んだ、粒度の高い描写が必要です。\n\n"

				"【出力形式（厳守）】\n"
				"回答は必ず以下のJSON形式のみを出力してください。\n"
				"各場所・施設名をキーとし、150文字以内の短文としてまとめてください。\n"
				"各場所につき、多様な観点から **20個** の短文リストを作成。\n\n"
				"Markdown記法（```json 等）は含めず、生のJSONテキストのみを返してください。\n\n"
                """
				{
					"入力された場所・施設名1": ["具体的なペルソナと行動1（情景＋属性＋感覚＋場所＋行為）", "具体的なペルソナと行動2（情景＋属性＋感覚＋場所＋行為）", ...],
					"入力された場所・施設名2": ["具体的なペルソナと行動3（情景＋属性＋感覚＋場所＋行為）", ...],
				}
				"""
				"\n\n"
				
				"【抽出・生成のルール】\n"
				"1. 語彙の解像度\n"
                "- 単語の羅列ではなく、情景が浮かぶ「属性＋具体的な状況＋行動」のセットで記述してください。\n"
				"- NG例: 「公園」「運動」「サラリーマン」\n"
				"- OK例: 「週末の親水公園で大型犬を遊ばせる愛犬家」「深夜の24時間ジムで大会に向けて追い込むトレーニー」「早朝の駅ビルでPC作業をするノマドワーカー」\n"
				"- NG例: 「人々が運動している」\n"
				"- OK例: 「ダンベルの金属音が響く中、鏡の前でフォームを確認するタンクトップの男性」\n"
				"- NG例: 「静かなカフェ」\n"
				"- OK例: 「エスプレッソマシンの蒸気音と、MacBookのキーボードを叩く音が混ざり合う」\n\n"

				"2. 多角的な視点を取り入れる（以下の要素をミックスする）\n"
				"- 【時間】: 早朝の静けさ、ランチタイムの行列、深夜の気配\n"
				"- 【感覚】: コーヒーの香り、塩素の匂い、エアコンの冷気、BGMの重低音\n"
				"- 【対人】: 一人での没頭、友人との談笑、家族連れの喧騒\n\n"

				"3. 文体および文量\n"
				"- 「〜です」「〜ます」は不要。体言止めや現在進行形で、情景を切り取るように記述する。\n"
				"- 1文あたり120文字程度を推奨。"
				"- 入力された場所・施設名ごとに20個づつ抽出してください。\n\n"
			))

async def fetch_analysis(semaphore:asyncio.Semaphore, user_msg:str, keys:List):
	max_retry = 5
	for idx in range(max_retry):
		idx_temperature = TEMPERATURE * (0.9 ** idx)
		try:
			async with semaphore:
				response     = await llmClient.chat.completions.create(
										messages=[system_msg, user_msg],
										tools=None,
										tool_choice=None,
										max_tokens=MAX_TOKENS,
										temperature=idx_temperature,
										top_p=TOP_P,
										model=AI_FOUNDRY_MODEL
									)
				raw_content  = response.choices[0].message.content

			content_dict = json.loads(raw_content)
			missing      = set(keys) - set(content_dict.keys())
			excess       = set(content_dict.keys()) - set(keys)
			
			if missing:
				raise ValueError(f"出力キーが不足しています。Missing: {missing}")
			if excess:
				for key in excess:
					del content_dict[key]

			return keys, content_dict
			
		except json.JSONDecodeError as e:
			print(f"[Attempt {idx+1}] Decode Error:: {e}")
			print(raw_content)
			print()

			# 一旦、サーバー負荷の軽減しつつリトライ
			await asyncio.sleep(1)
		
		except Exception as e:
			print(f"[Attempt {idx+1}] API Error: {e}")

			# 一旦、サーバー負荷の軽減しつつリトライ
			await asyncio.sleep(3)
	
	print(f"Failed all {max_retry} attempts for this batch.")
	return keys, {}

async def main():
    semaphore = asyncio.Semaphore(50)
    tasks     = []
    for idx in range(0, len(data_list), 3):
        set_place = data_list[idx : idx+3]
        user_msg  = UserMessage(content=json.dumps(set_place))
        tasks.append(fetch_analysis(semaphore, user_msg, set_place))
    
    results       = await tqdm_asyncio.gather(*tasks)
    analysis_data = {}
    danger_set    = set()
    for keys, result in results:
        if result == {}: danger_set.update(keys)
        else:            analysis_data |= result
    
    return analysis_data, danger_set

data_list                 = [dict_code2name[elem] for elem in np_codelist.tolist()]
analysis_data, danger_set = await main()
analysis_data

In [ ]:
async def correct_danger(danger_set:set):
    semaphore = asyncio.Semaphore(50)
    tasks     = []
    for elem in danger_set:
        user_msg = UserMessage(content=f"['{elem}']")
        tasks.append(fetch_analysis(semaphore, user_msg, [elem]))
    
    results      = await tqdm_asyncio.gather(*tasks)
    correct_data = {}
    true_danger  = set()
    for keys, result in results:
        if result == {}: true_danger.update(keys)
        else:            correct_data |= result
    
    return correct_data, true_danger

correct_data, true_danger = await correct_danger(danger_set)
correct_data

In [ ]:
# メモ：
# true_dangerがどうしても、発生してしまう
# このセルで手動で、問題の解決を図ること

final_data = analysis_data | correct_data
print(len(data_list))
print(len(final_data))
print(true_danger)
print()

# target   = '不動産取引・商業施設設計業'
# user_msg = UserMessage(content=f"['{target}']")
# print(await fetch_analysis(asyncio.Semaphore(1), user_msg, [target]))
# print()

# target   = '駐車場・駐輪場関連'
# user_msg = UserMessage(content=f"['{target}']")
# print(await fetch_analysis(asyncio.Semaphore(1), user_msg, [target]))
# print()

# target   = '自動車・オートバイ・自転車・ドライブ関連'
# user_msg = UserMessage(content=f"['{target}']")
# print(await fetch_analysis(asyncio.Semaphore(1), user_msg, [target]))
# print()

# JSON形式でバックアップ
with open('sentence-traindata.json', 'w', encoding='utf-8') as f:
    json.dump(final_data, f, indent=4, ensure_ascii=False)
with open('sentence-traindata.json', 'r', encoding='utf-8') as f:
    final_data = json.load(f)
final_data

In [ ]:
model         = SentenceTransformer('cl-nagoya/ruri-v3-130m')
count_list    = [len(final_data[key]) for key in final_data]
list_location = [scene                for key in final_data for scene in final_data[key]]
np_matrix     = model.encode(list_location, normalize_embeddings=True)  # (スポット数N × シーン数M) × 512

indices       = np.cumsum(count_list)[:-1]
final_matrix  = np.array([chunk.mean(axis=0) for chunk in np.split(np_matrix, indices)])
l2norm_vector = np.linalg.norm(final_matrix, axis=1, keepdims=True)
final_matrix  = final_matrix / np.maximum(l2norm_vector, 1e-10)
final_matrix

In [ ]:
# メモ：
# final_matrixの行部分の順番は、バラバラになっている
# これをnp_codelistと同期済みのdata_listを利用してソート・整形する

place_names   = {elem:idx for idx, elem in enumerate(data_list)}
np_places     = np.array([place_names[key] for key in final_data])

sort_indices  = np.argsort(np_places)
sorted_places = np.array(data_list)
sorted_matrix = final_matrix[sort_indices, :]
sorted_matrix = sorted_matrix.astype(np.float32)

NPZ_PATH = 'cohort_caption_matrix.npz'
np.savez_compressed(
    NPZ_PATH,
    data=sorted_matrix,
    business_placelist=sorted_places,
    dict_code2name=np.array(dict_code2name)
)

sorted_matrix